# **1. Overview**

## Description

**Why This Matters**

Accurate sales forecasts are crucial for planning process, supply chain processes, delivery logistics and inventory management. By optimizing forecasts, we can minimize waste and streamline operations, making our e-grocery services more sustainable and efficient.

**Your Impact**

Your participation in this challenge will directly contribute to Rohlik mission of sustainable and efficient e-grocery delivery. Your insights will help us enhance customer service and achieve a greener future.

We are relaunching the Challenge with prizes.

Rohlik Group, a leading European e-grocery innovator, is revolutionising the food retail industry. We operate across 11 warehouses in Czech Republic, Germany, Austria, Hungary, and Romania.

We are now transitioning from the Rohlik Orders Forecasting Challenge to the Rohlik Sales Forecasting Challenge, as we continue with our set of challenges. This challenge focuses on predicting the sales of each selected warehouse inventory for next 14 days using historical sales data.

## Evaluation

Submissions are evaluated on Weighted Mean Absolute Error (WMAE) between the predicted sales and the actual sales. Weights for the test evaluation can be found in the Data section.

## Submission File

For each ID in the test set, you must predict a probability for the TARGET variable. The file should contain a header and have the following format:

id,sales_hat

840_2024-06-10,12.01

2317_2024-06-15,13.32

738_2024-06-10,14.12

3894_2024-06-11,3.03

3393_2024-06-08,53.03


## Prizes
Leaderboard prizes

1st place - $4,000

2nd place - $4,000

3rd place - $2,000

## Citation

MichalKecera. Rohlik Sales Forecasting Challenge. https://kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2, 2024. Kaggle.

## Data

## Dataset Description
You are provided with historical sales data for selected Rohlik inventory and date. IDs, sales, total orders and price columns are altered to keep the real values confidential. Some features are not available in test as they are not known at the moment of making the prediction. The task is to forecast the sales column for a given id, constructed from unique_id and date (e. g. id 1226_2024-06-03 from unique_id 1226 and date 2024-06-03), for the test set.



## Files
- **sales_train.csv** - training set containing the historical sales data for given date and inventory with selected features described below
- **sales_test.csv** - full testing set
- **inventory.csv** - additional information about inventory like its product (same products across all warehouses share same product unique id and name, but have different unique id)
- **solution.csv** - full submission file in the correct format
- **calendar.csv** - calendar containing data about holidays or warehouse specific events, some columns are already in the train data but there are additional rows in this file for dates where some warehouses could be closed due to public holiday or Sunday (and therefore they are not in the train set)



## Columns
**sales_train.csv** and **sales_test.csv**

- `unique_id` - unique id for inventory
- `date` - date
- `warehouse` - warehouse name
- `total_orders` - historical orders for selected Rohlik warehouse known also for test set as a part of this challenge
- `sales` - Target value, sales volume (either in pcs or kg) adjusted by availability. The sales with lower availability than 1 are already adjusted to full potential sales by Rohlik internal logic. There might be missing dates both in train and test for a given inventory due to various reasons. This column is missing in test.csv as it is target variable.
- `sell_price_main` - sell price
- `availability` - proportion of the day that the inventory was available to customers. The inventory doesn't need to be available at all times. A value of 1 means it was available for the entire day. This column is missing in test.csv as it is not known at the moment of making the prediction.
- `type_0_discount`, type_1_discount, … - Rohlik is running different types of promo sale actions, these show the percentage of the original price during promo ((original price - current_price) / original_price). Multiple discounts with different type can be run at the same time, but always the highest possible discount among these types is used for sales. Negative discount value should be interpreted as no discount.

**inventory.csv**

- `unique_id` - inventory id for a single keeping unit
- `product_unique_id` - product id, inventory in each warehouse has the same product unique id (same products across all warehouses has the same product id, but different unique id)
- `name` - inventory id for a single keeping unit
L1_category_name, L2_category_name, … - name of the internal category, the higher the number, the more granular information is present
- `warehouse` - warehouse name

**calendar.csv**

- `warehouse` - warehouse name
- `date` - date
- `holiday_name` - name of public holiday if any
- `holiday` - 0/1 indicating the presence of holidays
- `shops_closed` - public holiday with most of the shops or large part of shops closed
- `winter_school_holidays` - winter school holidays
- `school_holidays` - school holidays

**test_weights.csv**

- `unique_id` - inventory id for a single keeping unit
- `weight` - weight used for final metric computation

# **Import and merge DFs**

In [30]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Define the correct path using forward slashes
path = r"C:\\Users\defaultuser0\\OneDrive\Documents\\GitHub\\CapstoneProject-RohlikSalesForecasting\\rohlik-sales-forecasting-challenge-v2\Data"

# Load cleaned datasets
df = pd.read_csv(fr"{path}\dataframes\df_cleaned.csv")
sales_train = pd.read_csv(fr"{path}\sales_train.csv", index_col=False)
sales_test = pd.read_csv(fr"{path}\sales_test.csv", index_col=False)
weights = pd.read_csv(fr"{path}\test_weights.csv", index_col=False)
solution = pd.read_csv(fr"{path}\solution.csv", index_col=False)
inventory = pd.read_csv(fr"{path}\inventory.csv", index_col=False)
calendar = pd.read_csv(fr"{path}\calendar.csv", index_col=False)
frankfurt_weather = pd.read_csv(fr"{path}\weather\frankfurt_weather.csv", index_col=False)
brno_weather = pd.read_csv(fr"{path}\weather\brno_weather.csv", index_col=False)
budapest_weather = pd.read_csv(fr"{path}\weather\budapest_weather.csv", index_col=False)
munich_weather = pd.read_csv(fr"{path}\weather\munich_weather.csv", index_col=False)
prague_weather = pd.read_csv(fr"{path}\weather\prague_weather.csv", index_col=False)

# **Feature Engineering**

In [31]:
"""
Date Features
"""
# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])
sales_train['date'] = pd.to_datetime(sales_train['date'])
sales_test['date'] = pd.to_datetime(sales_test['date'])

# Extract date features
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['week_of_year'] = df['date'].dt.isocalendar().week
df['week_of_month'] = ((df['date'].dt.day - 1) // 7) + 1
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_of_week
df['day_of_year'] = df['date'].dt.day_of_year 

"""
sin and cos to capture cyclical or seasonal patterns in time-related data
"""

# day
df['dayofweek_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['dayofmonth_sin'] = np.sin(2 * np.pi * df['day'] / 31)
df['dayofmonth_cos'] = np.cos(2 * np.pi * df['day'] / 31)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365) # Correlation 0.97 with 'month_sin'
df['dayofyear_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365) # Correlation 0.95 with 'month_cos'

# week
df['weekofyear_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52) # Correlation 1 with 'dayofyear_sin'
df['weekofyear_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52) # Correlation 0.96 with 'month_cos'
df['weekofmonth_sin'] = np.sin(2 * np.pi * df['week_of_month'] / 4)
df['weekofmonth_cos'] = np.cos(2 * np.pi * df['week_of_month'] / 4)

# month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12) # High correlation with multiple features
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# quarter
df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)

# drop month: High correlation with multiple features in past tests
df.drop(columns=['quarter'], axis=1, inplace=True)

In [32]:
"""
Treating `discount` columns as instructed in Overview
"""
# Create discount column by selecting max discount value across discount columns
discount_cols = [col for col in df.columns if col.startswith('type_') and col.endswith('_discount')]
df['discount'] = df[discount_cols].max(axis=1)

# drop discount columns
df.drop(columns = ['type_0_discount', 'type_1_discount', 'type_2_discount',
                            'type_3_discount', 'type_4_discount', 'type_5_discount',
                            'type_6_discount'], inplace = True)

In [33]:
""" 
Merging Weather info
"""

# sorting the dataframe by date
df.sort_values('date', inplace=True)

# mapping cities
df['city'] = df['warehouse'].map({'Frankfurt_1': 'Frankfurt',
                                  'Brno_1': 'Brno',
                                  'Budapest_1': 'Budapest',
                                    'Munich_1': 'Munich',
                                   'Prague_1': 'Prague',
                                   'Prague_2': 'Prague',
                                   'Prague_3': 'Prague'})

# List of weather DataFrames and corresponding city names
weather_dfs = [
    (frankfurt_weather, 'Frankfurt'),
    (brno_weather, 'Brno'),
    (budapest_weather, 'Budapest'),
    (munich_weather, 'Munich'),
    (prague_weather, 'Prague')
]

for df_weather, city_name in weather_dfs:
    df_weather.rename(columns={"datetime": "date"}, inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'])
    df_weather['city'] = city_name
    
# Concatenate all weather data into one
weather_df_list = [frankfurt_weather, brno_weather, budapest_weather, munich_weather, prague_weather]
full_weather_df = pd.concat(weather_df_list, ignore_index=True)

# Rename columns for consistency
full_weather_df.rename(columns={"name": "latitudelongitude"}, inplace=True)

# Final merge
df = df.merge(full_weather_df, on=['date', 'city'], how='left')

In [34]:
""" 
Concatenaing category columns
"""
df['category'] = (
    df['L1_category_name_en'].astype(str) + "_" +
    df["L2_category_name_en"].str.split('_').str[-1] + "_" +
    df["L3_category_name_en"].str.split('_').str[-1] + "_" +
    df["L4_category_name_en"].str.split('_').str[-1]
)

df.drop(columns=["L1_category_name_en",
                    "L2_category_name_en",
                    "L3_category_name_en",
                    "L4_category_name_en"],
        inplace=True
        )

In [35]:
""" 
Calculates days to shops_closed
"""

# Forward fill the dates where shops are closed (i.e., use the last closure date)
df['next_closure_date'] = df['date'].where(df['shops_closed'] == 1).bfill()

# Calculate the number of days after the last shop closure
df['days_next_closure'] = (df['next_closure_date'] - df['date']).dt.days

# fill NaN values with 0
df['days_next_closure'].fillna(0, inplace=True)

# drop next closure
df.drop(columns=['next_closure_date'], inplace=True)

In [36]:
""" 
Calculates days after shops_closed and day_after_closing
"""

# Forward fill the dates where shops are closed (i.e., use the last closure date)
df['last_closure_date'] = df['date'].where(df['shops_closed'] == 1).ffill()

# Calculate the number of days after the last shop closure
df['days_after_closure'] = (df['date'] - df['last_closure_date']).dt.days

# fill NaN values with 0
df['days_after_closure'].fillna(0, inplace=True)

# Bool value for shops_closed eve
df['day_after_closing'] = np.where(df['days_after_closure'] == 1, 1, 0)

# drop next closure
df.drop(columns=['last_closure_date'], inplace=True)

In [37]:
""" 
Mean availability
"""

# Filter data before the given date
filtered_df = df[df['date'] <= sales_train['date'].max()]

# Group by 'unique_id' and calculate mean availability
mean_availability = filtered_df.groupby('unique_id')['availability'].mean().reset_index()

# Rename the resulting column
mean_availability.rename(columns={'availability': 'mean_availability'}, inplace=True)

# Merge mean availability with the original DataFrame
df = df.merge(mean_availability, on='unique_id', how='left')

# Drop availability column
df.drop(columns=['availability'], axis=1, inplace=True)


### [Target encoding categorical variables](https://mlbook.explained.ai/catvars.html#target-encoding)

In [38]:
""" 
Mean Sales per unique_id
"""

# Filter data before the given date
filtered_df = df[df['date'] <= sales_train['date'].max()]

# Group by 'unique_id' and calculate the mean sales
mean_sales= filtered_df.groupby('unique_id')['sales'].mean().reset_index()

# Rename the resulting column
mean_sales.rename(columns={'sales': 'mean_sales'}, inplace=True)

# merging
df = df.merge(mean_sales, on='unique_id', how='left')

### Rolling Features

In [39]:
# # Create rolling features
# daily_sales = df_frank.groupby("date")["mean_sales"].sum().reset_index().rename(columns={"mean_sales":"daily_mean_sales_sum"})
# daily_sales_mean = df_frank.groupby("date")["mean_sales"].mean().reset_index().rename(columns={"mean_sales":"daily_mean_sales_mean"})
# daily_sales_std= df_frank.groupby("date")["mean_sales"].std().reset_index().rename(columns={"mean_sales":"daily_mean_sales_std"})

# # Aggregate info
# daily_sales = daily_sales.merge(daily_sales_mean, on='date', how='left')
# daily_sales = daily_sales.merge(daily_sales_std, on='date', how='left')

# # Step 2: Calculate rolling statistics globally
# for i in [1, 3, 7, 15, 30, 60, 90, 120, 180, 365]:
#     daily_sales[f"Rolling_{i}D_Mean"] = daily_sales["daily_mean_sales_sum"].rolling(window=i, min_periods=1).mean()
#     daily_sales[f"Rolling_{i}D_Sum"] = daily_sales["daily_mean_sales_sum"].rolling(window=i, min_periods=1).sum()
    
# df_frank = df_frank.merge(daily_sales, on="date", how="left")

### Lag Features

In [40]:
# Create lag features for 1-day, 2-day, and 3-day previous sales
daily_lag_sales = df.groupby("date")['mean_sales'].sum().shift(1).reset_index().rename(columns={'mean_sales':'lag_1D'})
daily_lag_sales_2D = df.groupby("date")['mean_sales'].sum().shift(2).reset_index().rename(columns={'mean_sales':'lag_2D'})
daily_lag_sales_3D = df.groupby("date")['mean_sales'].sum().shift(3).reset_index().rename(columns={'mean_sales':'lag_3D'})

# Aggregate results
daily_lag_sales = daily_lag_sales.merge(daily_lag_sales_2D, on = 'date', how='left')
daily_lag_sales = daily_lag_sales.merge(daily_lag_sales_3D, on = 'date', how='left')

# Merge DFs
df = df.merge(daily_lag_sales, on='date', how='left')   

### Python Feature Engineering Cookbook - Second Edition

[Transforming variables with the logarithm function](https://learning-oreilly-com.stclair.idm.oclc.org/library/view/python-feature-engineering/9781804611302/B18894_03.xhtml#_idParaDest-90)

*To access this book, make sure to Sign in the MyStClairCollege Library Search OR and O'REILLY active subscription*

In [41]:
""" 
Check and deal with skewness 
"""

# bin_columns = ["holiday", "winter_school_holidays", "school_holidays"]



# # Check skewness for all numeric columns
# skewness = df_frank.drop(columns=bin_columns, axis = 1).skew(numeric_only=True)

# # Identify features with high skewness (threshold > 0.5 or < -0.5)
# high_skew = skewness[abs(skewness) > 0.5]
# print(high_skew)

# # # Transforming values
# df_frank[high_skew.index] = np.log1p(df_frank[high_skew.index])

' \nCheck and deal with skewness \n'

In [42]:
df.to_csv(f"{path}\dataframes\df_engineered.csv", index_label=False)